In [1]:
!python3 --version

Python 3.7.16


In [2]:
import torch
torch.cuda.set_device(0)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 0
Current default GPU name: NVIDIA A40


In [3]:
from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image
from diffusers.utils import load_image

import torch

pipe = AutoPipelineForImage2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")


The config attributes {'feature_extractor': [None, None], 'image_encoder': [None, None]} were passed to StableDiffusionXLImg2ImgPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'feature_extractor': [None, None], 'image_encoder': [None, None]} are not expected by StableDiffusionXLImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'reverse_transformer_layers_per_block': None} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


StableDiffusionXLImg2ImgPipeline {
  "_class_name": "StableDiffusionXLImg2ImgPipeline",
  "_diffusers_version": "0.21.4",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "force_zeros_for_empty_prompt": true,
  "requires_aesthetics_score": false,
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

## image generation with sdxl_turbo

In [4]:
# dataset
import torchvision

from avalanche.benchmarks import SplitMNIST, SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger, TensorboardLogger

from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training.plugins import ReplayPlugin
from types import SimpleNamespace
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy

# all imports

import torch
import os
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import datasets, transforms
import torch.optim.lr_scheduler # ?
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop, RandomHorizontalFlip, Resize
from torchvision.transforms.functional import center_crop
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.utils import save_image
from torchvision.transforms.functional import pil_to_tensor


In [5]:
# !pip install torchvision

In [6]:
# trainset = torchvision.datasets.CIFAR100(root='data', train=True,
#                                          download=True)

# name_list = trainset.classes

# integer_to_name = {i: name for i, name in enumerate(name_list)}


In [7]:
def save_specific_cifar100(dataset, num_images_per_class, save_dir, real_list):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    torch.manual_seed(41)

    saved_counts = {label: 0 for label in range(100)}  # Initialize saved image count for each class

    transform_to_tensor = transforms.ToTensor()

    indices = torch.randperm(len(dataset)).tolist()
    
    for idx in indices:
        image, label = dataset[idx]
        
        if label in real_list:
            # Check if image is already a tensor, transform if not
            if not isinstance(image, torch.Tensor):
                image_tensor = transform_to_tensor(image)
            else:
                image_tensor = image  # Use the image directly if it is already a tensor

            # Skip saving if this class already has the desired number of images saved
            if saved_counts[label] >= num_images_per_class:
                continue

            class_name = dataset.classes[label]
            image_path = os.path.join(save_dir, f'{class_name}{saved_counts[label]}.png')
            save_image(image_tensor, image_path)
            saved_counts[label] += 1

            # Save path to file
            class_file_path = os.path.join(save_dir, f"class{label}.txt")
            with open(class_file_path, "a") as file:
                file.write(f"{image_path} {label}\n")

            # Check if we have finished saving the specified number of images for all classes
            if all(count >= num_images_per_class for count in saved_counts.values()):
                break

    print(f"Saved {num_images_per_class} images per class from the CIFAR-100 training dataset.")

def count_txt_files(directory):
    """
    Counts the number of .txt files in the specified directory.

    Args:
    directory (str): The path to the directory to search for .txt files.

    Returns:
    int: The number of .txt files in the directory.
    """
    txt_count = 0
    # List all files and directories in the specified directory
    for entry in os.listdir(directory):
        # Construct full path
        full_path = os.path.join(directory, entry)
        # Check if it's a file with a .txt extension
        if os.path.isfile(full_path) and entry.endswith('.txt'):
            txt_count += 1
    
    return txt_count

def count_png_files(folder_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print("The specified folder does not exist.")
        return 0
    
    # List all files in the directory
    files = os.listdir(folder_path)
    
    # Filter and count files that end with .txt
    txt_files_count = sum(1 for file in files if file.endswith('.png'))
    
    return txt_files_count

In [8]:
benchmark = SplitCIFAR100(n_experiences=20,
                          seed = 41,             
                          )

Files already downloaded and verified
Files already downloaded and verified


In [9]:
orders = benchmark.classes_order
order_list = [orders[x:x+5] for x in range(0, len(orders), 5)]
print(order_list)

order_sample = [order[3:] for order in order_list]


[[36, 0, 54, 5, 20], [22, 45, 13, 83, 19], [26, 73, 16, 62, 33], [34, 98, 24, 74, 53], [10, 94, 51, 4, 32], [38, 81, 50, 40, 41], [30, 89, 69, 64, 21], [84, 14, 88, 49, 68], [6, 80, 57, 65, 46], [9, 91, 48, 72, 31], [76, 7, 47, 8, 1], [61, 75, 63, 18, 86], [59, 70, 43, 85, 95], [27, 93, 35, 25, 82], [44, 56, 67, 66, 37], [60, 11, 2, 78, 52], [97, 39, 55, 3, 99], [29, 71, 23, 28, 90], [87, 15, 92, 17, 77], [12, 42, 96, 79, 58]]


In [10]:
import json
import os

def load_json(file_path):
    """
    Load a JSON file and return the data.

    :param file_path: Path to the JSON file.
    :return: Parsed JSON data.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_labels(json_data):
    """
    Process the labels from the JSON data.

    :param json_data: The JSON data containing labels and class names.
    """
    for identifier, class_name in json_data.items():
        print(f"ID: {identifier}, Class Name: {class_name}")

def replace_spaces_with_underscores(class_names):
    modified_names = []
    for name in class_names:
        # Replace spaces with underscores
        modified_name = name.replace(' ', '_')
        modified_names.append(modified_name)
    return modified_names



In [11]:
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100

stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform = transform_train = Compose([
    # Resize(224),
    # Resize(384),
    # RandomHorizontalFlip(),
    ToTensor(),
    # Normalize(*stats,inplace=True)
])

# Load the CIFAR-100 training set
trainset = torchvision.datasets.CIFAR100(root='data', train=True,download=True, transform=transform)

name_list = trainset.classes
integer_to_name = {i: name_list[i] for i in range(100)}
syn_classes = [5, 20, 83, 19, 62, 33, 74, 53, 4, 32, 40, 41, 64, 21, 49, 68, 65, 46, 72, 31, 8, 1, 18, 86, 85, 95, 25, 82, 66, 37, 78, 52, 3, 99, 28, 90, 17, 77, 79, 58]
real_classes = list(set([i for i in range(100)]) - set(syn_classes))
print(len(real_classes))

# sythnthesis classes
benchmark = SplitCIFAR100(n_experiences=20,
                          seed = 41,             
                          )

orders = benchmark.classes_order
order_list = [orders[x:x+5] for x in range(0, len(orders), 5)]

order_sample = [order[3:] for order in order_list]
classname_list = []
label_list = []
classname_list_sep = []
for order_l in order_sample:
    label_list.append(order_l)
    cur_classname = [integer_to_name[i] for i in order_l]
    classname_list.append(cur_classname)
classname_list_sep = [item for lists in classname_list for item in lists]
label_list_sep = [item for lists in label_list for item in lists]
print(label_list_sep)

Files already downloaded and verified
60
Files already downloaded and verified
Files already downloaded and verified
[5, 20, 83, 19, 62, 33, 74, 53, 4, 32, 40, 41, 64, 21, 49, 68, 65, 46, 72, 31, 8, 1, 18, 86, 85, 95, 25, 82, 66, 37, 78, 52, 3, 99, 28, 90, 17, 77, 79, 58]


In [12]:
real_to_syn_id = {5: 87, 20: 84, 83: 51, 19: 89, 62: 70, 33: 59, 74: 63, 53: 16, 4: 55, 32: 26, 40: 9, 41: 89, 64: 24, 21: 29, 49: 12, 68: 12, 65: 80, 46: 11, 72: 9, 31: 42, 8: 48, 1: 26, 18: 24, 86: 22, 85: 9, 95: 73, 25: 87, 82: 92, 66: 88, 37: 94, 78: 44, 52: 47, 3: 97, 99: 75, 28: 9, 90: 81, 17: 22, 77: 26, 79: 80, 58: 89}

In [13]:
# output_folder = 'saved_data/cifar_train500/'
# count = count_txt_files(output_folder)
# print(f"There are {count} .txt files in the folder.")
# count = count_png_files(output_folder)
# print(f"There are {count} .png files in the folder.")

# save_specific_cifar100(trainset, 500, 'saved_data/cifar_train500_2real_fori2i', label_list_sep)

In [14]:
def count_txt_files(folder_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print("The specified folder does not exist.")
        return 0
    
    # List all files in the directory
    files = os.listdir(folder_path)
    
    # Filter and count files that end with .txt
    txt_files_count = sum(1 for file in files if file.endswith('.txt'))
    
    return txt_files_count

# Specify the folder path here
folder_path = 'saved_data/cifar_train500_2real_fori2i'

# Call the function and print the result
count = count_txt_files(folder_path)
print(f"There are {count} .txt files in the folder.")

The specified folder does not exist.
There are 0 .txt files in the folder.


In [15]:
import random

def extract_prompts_fromtxts(file_path):
    prompts = []  # List to hold the prompts
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip() != 'USER':  # Ignore lines that are just 'USER'
                # Split the line at the first colon and strip whitespace from the prompt
                parts = line.split(':', 1)
                if len(parts) > 1:
                    prompt = parts[1].strip()
                    prompts.append(prompt)
    return prompts


def create_sdxl_data_fixed_prompts_randommultiple(class_dict, image_size, prompt_file_dict, generator_seed, num_image_replay=50, folder_name="/content/sd_images"):
    # Create the folder if it doesn't exist
    random.seed(42)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        print("Generating images for class " + str(id) + ": " + class_name)

        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        existing_images_count = 0

        # Check how many images already exist for the class
        if os.path.exists(class_file_path):
            with open(class_file_path, "r") as file:
                existing_images_count = len(file.readlines())
            num_images_to_generate = num_image_replay - existing_images_count
            
            if existing_images_count >= num_image_replay:
                print(f"Class {class_name} already contains {num_image_replay} images. Moving to the next class.")
                continue  # Move to the next class if enough images already exist
        prompt_file_path = os.path.join(prompt_file_dict, f"class{id}.txt")
        current_prompt_list = extract_prompts_fromtxts(prompt_file_path)
        
        indices = [random.randint(0, len(current_prompt_list)-1) for _ in range(num_image_replay)]
        current_prompt_list = [current_prompt_list[item] for item in indices]

        # Generate the new images
        with open(class_file_path, "a") as file:
            for j, prompt in enumerate(current_prompt_list):
                print(prompt)
                image_name = class_name + f"{j}.png"
                image_path = os.path.join(folder_name, image_name)
                new_image = pipe(prompt=prompt, generator = generator_seed, num_inference_steps=1, guidance_scale=1.0).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(image_path)

                file.write(f"{image_path} {id}\n")

                print(f"Generated image {image_name} for class {class_name}")


def create_sdxl_data_fixed_prompts(class_dict, image_size, prompt_file_dict, generator_seed, num_image_replay=50, folder_name="/content/sd_images"):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        print("Generating images for class " + str(id) + ": " + class_name)

        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        existing_images_count = 0

        # Check how many images already exist for the class
        if os.path.exists(class_file_path):
            with open(class_file_path, "r") as file:
                existing_images_count = len(file.readlines())
            num_images_to_generate = num_image_replay - existing_images_count
            
            if existing_images_count >= num_image_replay:
                print(f"Class {class_name} already contains {num_image_replay} images. Moving to the next class.")
                continue  # Move to the next class if enough images already exist
        prompt_file_path = os.path.join(prompt_file_dict, f"class{id}.txt")
        current_prompt_list = extract_prompts_fromtxts(prompt_file_path)

        # Generate the new images
        with open(class_file_path, "a") as file:
            for j, prompt in enumerate(current_prompt_list):
                print(prompt)
                image_name = class_name + f"{j}.png"
                image_path = os.path.join(folder_name, image_name)
                new_image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=1.0).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(image_path)

                file.write(f"{image_path} {id}\n")

                print(f"Generated image {image_name} for class {class_name}")
                
                
                
def create_sdxl_data_fixed_prompts_randommultiple_img2img(class_dict, image_size, prompt_file_dict, startimage_file_path,
                                                          generator_seed, num_image_replay=50, folder_name="/content/sd_images"):
    
    """
    not done, todo: provide image list as current_prompt_list
    """
    # Create the folder if it doesn't exist
#     random.seed(42)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        print("Generating images for class " + str(id) + ": " + class_name)
        
        path_file = os.path.join(startimage_file_path, f"class{id}.txt")
        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        with open(path_file, 'r') as file:
            startimage_file_paths = [line.split()[0] for line in file.readlines()]
        existing_images_count = 0

        # Check how many images already exist for the class
        if os.path.exists(class_file_path):
            with open(class_file_path, "r") as file:
                existing_images_count = len(file.readlines())
            num_images_to_generate = num_image_replay - existing_images_count
            
            if existing_images_count >= num_image_replay:
                print(f"Class {class_name} already contains {num_image_replay} images. Moving to the next class.")
                continue  # Move to the next class if enough images already exist
        prompt_file_path = os.path.join(prompt_file_dict, f"class{id}.txt")
        current_prompt_list = extract_prompts_fromtxts(prompt_file_path)
        
        random.seed(42)
        indices1 = [random.randint(0, len(current_prompt_list)-1) for _ in range(num_image_replay)]
        current_prompt_list = [current_prompt_list[item] for item in indices1]

        random.seed(41)
        indices2 = [random.randint(0, len(startimage_file_paths)-1) for _ in range(num_image_replay)]
        current_init_image_list = [startimage_file_paths[item] for item in indices2]

        with open(class_file_path, "a") as file:
            for j, prompt in enumerate(current_prompt_list):
#                 print(prompt)
                
                starting_image_path = current_init_image_list[j]
                init_image = load_image(starting_image_path).resize((512, 512))

                image_name = starting_image_path.split('/')[-1]
                image_path = os.path.join(folder_name, image_name)
                new_image = pipe(prompt=prompt,image = init_image, 
                                strength = 0.5, 
                                num_inference_steps=40, 
                                generator = generator_seed, guidance_scale=2).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(image_path)

                file.write(f"{image_path} {id}\n")

            print(f"Generated images for class {class_name}")

In [16]:
def create_sdxl_data_img2img_matching_image_prompt(class_dict, image_size, prompt_file_dict, startimage_file_path,
                                                          generator_seed, num_image_replay=50, folder_name="/content/sd_images"):
    
    """
    this is specified to matching pair of lavva prompt and images
    """
    # Create the folder if it doesn't exist
    random.seed(42)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        print("Generating images for class " + str(id) + ": " + class_name)

        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        existing_images_count = 0

        # Check how many images already exist for the class
        if os.path.exists(class_file_path):
            with open(class_file_path, "r") as file:
                existing_images_count = len(file.readlines())
            num_images_to_generate = num_image_replay - existing_images_count
            
            if existing_images_count >= num_image_replay:
                print(f"Class {class_name} already contains {num_image_replay} images. Moving to the next class.")
                continue  # Move to the next class if enough images already exist
        prompt_file_path = os.path.join(prompt_file_dict, f"class{id}.txt")
        current_prompt_list = extract_prompts_fromtxts(prompt_file_path)

        # Generate the new images
        with open(class_file_path, "a") as file:
            for j, prompt in enumerate(current_prompt_list):
                print(prompt)
                image_name = class_name + f"{j}.png"
                
                starting_image_path = os.path.join(startimage_file_path, image_name)
                init_image = load_image(starting_image_path).resize((512, 512))

            
                image_path = os.path.join(folder_name, image_name)
                new_image = pipe(prompt=prompt,image = init_image, 
                                strength = 0.5, 
                                num_inference_steps=4, 
                                generator = generator_seed, guidance_scale=0.5).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(image_path)

                file.write(f"{image_path} {id}\n")

                print(f"Generated image {image_name} for class {class_name}")

In [17]:
def create_sdxl_llava_random(class_dict, image_size, generator_seed, num_image_replay=50, prompt_folder = "saved_data/llava_saved_data_310/llava_prompt_50", folder_name="/content/sd_images"):
    random.seed(42)
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        print("Generating images for class " + str(id) + ": " + class_name)

        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        existing_images_count = 0

        # Check how many images already exist for the class
        if os.path.exists(class_file_path):
            with open(class_file_path, "r") as file:
                existing_images_count = len(file.readlines())
            num_images_to_generate = num_image_replay - existing_images_count
            
            if existing_images_count >= num_image_replay:
                print(f"Class {class_name} already contains {num_image_replay} images. Moving to the next class.")
                continue  # Move to the next class if enough images already exist

        indices = [random.randint(0, 9) for _ in range(num_image_replay)]  # Assuming there are 10 prompts
        prompts = generate_prompts(class_name, indices)

        # Generate the new images
        with open(class_file_path, "a") as file:
            for j, prompt in enumerate(prompts):
                print(prompt)
                image_name = class_name + f"{j}.png"
                image_path = os.path.join(folder_name, image_name)
                new_image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=1.0).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(image_path)

                file.write(f"{image_path} {id}\n")

                print(f"Generated image {image_name} for class {class_name}")



In [23]:
def extract_prompts_fromtxts_real2syn(file_path):
    path = []
    prompts = []  # List to hold the prompts
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split(':', 1)
            if len(parts) > 1:
                img_path = parts[0].strip()
                prompt = parts[1].strip()
                path.append(img_path)
                prompts.append(prompt)
    return path, prompts

def create_sdxl_data_img2img_matching_image_prompt_real2syn(class_dict, image_size, prompt_file_dict, startimage_file_path, integer_to_name, dict_syn_to_real_id,
                                                          generator_seed, num_image_replay=50, folder_name="/content/sd_images"):
    
    """
    this is specified to matching pair of lavva prompt and images
    """
    # Create the folder if it doesn't exist
    random.seed(42)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for id, class_name in class_dict.items():
        # id, class_name are synthetic id and classname
        print("Generating images for class " + str(id) + ": " + class_name)

        class_file_path = os.path.join(folder_name, f"class{id}.txt")
        existing_images_count = 0

        # Check how many images already exist for the class
        
        num_images_to_generate = num_image_replay
        
        real_id = dict_syn_to_real_id[id] # this is the real class id that bert thinks is most similar to the current syn class
        real_class_name = integer_to_name[real_id]
        prompt_file_path = os.path.join(prompt_file_dict, f"class{real_id}.txt")
        current_path_list, current_prompt_list = extract_prompts_fromtxts_real2syn(prompt_file_path)

        # Generate the new images
        with open(class_file_path, "w") as file:
            for j, prompt in enumerate(current_prompt_list):
                # print(prompt)
                syn_prompt = prompt.replace(real_class_name, class_name)
                # print(prompt)
                syn_image_name = class_name + f"{j}.png"
                # real_image_name = real_class_name + f"{j}.png"
                # starting from real image
                # starting_image_path = os.path.join(startimage_file_path, real_image_name)
                starting_image_path = current_path_list[j]
                init_image = load_image(starting_image_path).resize((512, 512))

                # generated image is synthetic name
                generated_image_path = os.path.join(folder_name, syn_image_name)
                # int(num_inference_steps * strength)
                new_image = pipe(prompt=syn_prompt,image = init_image, 
                                strength = 1, 
                                num_inference_steps=10, 
                                generator = generator_seed, guidance_scale=8).images[0]
                resized_image = new_image.resize(image_size)

                resized_image.save(generated_image_path)

                file.write(f"{generated_image_path} {id}\n")

                print(f"Generated image {syn_image_name} for class {class_name}")

In [24]:
specific_integer_to_name = {key: integer_to_name[key] for key in label_list_sep if key in integer_to_name}

In [25]:
print(specific_integer_to_name)
print(len(specific_integer_to_name))

{5: 'bed', 20: 'chair', 83: 'sweet_pepper', 19: 'cattle', 62: 'poppy', 33: 'forest', 74: 'shrew', 53: 'orange', 4: 'beaver', 32: 'flatfish', 40: 'lamp', 41: 'lawn_mower', 64: 'possum', 21: 'chimpanzee', 49: 'mountain', 68: 'road', 65: 'rabbit', 46: 'man', 72: 'seal', 31: 'elephant', 8: 'bicycle', 1: 'aquarium_fish', 18: 'caterpillar', 86: 'telephone', 85: 'tank', 95: 'whale', 25: 'couch', 82: 'sunflower', 66: 'raccoon', 37: 'house', 78: 'snake', 52: 'oak_tree', 3: 'bear', 99: 'worm', 28: 'cup', 90: 'train', 17: 'castle', 77: 'snail', 79: 'spider', 58: 'pickup_truck'}
40


In [26]:
# import os

# def list_txt_files(folder_path):
#     # Check if the provided folder path exists
#     if not os.path.isdir(folder_path):
#         print(f"The folder path '{folder_path}' does not exist.")
#         return

#     # List all files in the folder
#     files = os.listdir(folder_path)
    
#     # Filter out .txt files
#     txt_files = [file for file in files if file.endswith('.txt')]
    
#     # Print the .txt files
#     if txt_files:
#         print("TXT files in the folder:")
#         for txt_file in txt_files:
#             print(txt_file)
#         print(len(txt_files))
#     else:
#         print("No TXT files found in the folder.")

# # Replace 'your_folder_path_here' with the actual folder path
# folder_path = 'saved_data/ImageNet/ImageNet_sdxl_llavaprompt_1300_3real_i2i'
# list_txt_files(folder_path)


## mixed data llava multiple generation image2image 500 real2syn

## strength 1 guide 8

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(42)
# create_sdxl_data_fixed_prompts_randommultiple(integer_to_name, image_size = (32,32), prompt_file_dict = 'saved_data/llava_saved_data_310/llava_prompt_50/', generator_seed = generator1, num_image_replay=50, folder_name='saved_data/sd_turbo_500images_llava_firstthreeclasses/')

create_sdxl_data_img2img_matching_image_prompt_real2syn(specific_integer_to_name, image_size = (32,32),
                                               prompt_file_dict = 'saved_data/llava_cifar100_real60_500/',
                                startimage_file_path = 'saved_data/cifar_train_all_fortest',
                                integer_to_name = integer_to_name,
                                dict_syn_to_real_id = real_to_syn_id,
                                generator_seed = generator1, num_image_replay=500, 
                                folder_name='saved_data/sdxl_llava_synfromreal_s1g8') # starting image and prompt from the similar class within the same experience

Token indices sequence length is longer than the specified maximum sequence length for this model (88 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['lost in thought while standing in front of the bed.']


Generating images for class 5: bed


Token indices sequence length is longer than the specified maximum sequence length for this model (88 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['lost in thought while standing in front of the bed.']


  0%|          | 0/10 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['create a relaxing and enjoyable atmosphere.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['create a relaxing and enjoyable atmosphere.']


...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']


Generated image cattle191.png for class cattle


  0%|          | 0/10 [00:00<?, ?it/s]

Generated image cattle192.png for class cattle


  0%|          | 0/10 [00:00<?, ?it/s]

## llava prompt multiple generation image2image imagenet

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(42)

# create_sdxl_data_fixed_prompts_randommultiple_img2img(specific_integer_to_name, image_size = (224,224),
                                               prompt_file_dict = 'saved_data/ImageNet/prompt_130_1/',
                                               startimage_file_path = 'saved_data/ImageNet/ImageNet_train_renamed_10percent/',
                                               generator_seed = generator1, num_image_replay=1300, 
                                               folder_name='saved_data/ImageNet/ImageNet_sdxl_llavaprompt_3real_i2i')

## mixed data llava multiple generation image2image 

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(42)
# create_sdxl_data_fixed_prompts_randommultiple(integer_to_name, image_size = (32,32), prompt_file_dict = 'saved_data/llava_saved_data_310/llava_prompt_50/', generator_seed = generator1, num_image_replay=50, folder_name='saved_data/sd_turbo_500images_llava_firstthreeclasses/')

create_sdxl_data_img2img_matching_image_prompt(integer_to_name, image_size = (32,32),
                                               prompt_file_dict = 'saved_data/llava_saved_data_310/llava_prompt_50/',
                                               startimage_file_path = 'saved_data/llava_saved_data_310/cifar_50/',
                                               generator_seed = generator1, num_image_replay=50, 
                                               folder_name='saved_data/sd_turbo_i2i_50all')

## mixed data llava multiple generation

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(42)
create_sdxl_data_fixed_prompts_randommultiple(specific_integer_to_name, image_size = (32,32), prompt_file_dict = 'saved_data/llava_saved_data_310/llava_prompt_50/', generator_seed = generator1, num_image_replay=500, folder_name='saved_data/sd_turbo_500images_llava_firstthreeclasses/')

## diverse prompts

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(41)
create_sdxl_data(specific_integer_to_name, image_size = (32,32), generator_seed = generator1, num_image_replay=500, folder_name='saved_data/sd_turbo_500images/')


## base prompt

In [ ]:
generator1 = torch.Generator(device="cuda").manual_seed(41)
create_sdxl_data_baseprompt(specific_integer_to_name, image_size = (32,32), generator_seed = generator1, num_image_replay=500, folder_name='saved_data/sd_turbo_500images_baseprompt/')


In [ ]:
import os

def count_txt_files(folder_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print("The specified folder does not exist.")
        return 0
    
    # List all files in the directory
    files = os.listdir(folder_path)
    
    # Filter and count files that end with .txt
    txt_files_count = sum(1 for file in files if file.endswith('.txt'))
    
    return txt_files_count

# Specify the folder path here
folder_path = 'saved_data/cifar_train500_2syn_i2i_step10/'

# Call the function and print the result
count = count_txt_files(folder_path)
print(f"There are {count} .txt files in the folder.")


In [ ]:
def read_and_print_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()  # Read all lines in the file
            print(len(lines))
            for line in lines:
                print(line.strip())  # Print each line, stripping newline characters
    except FileNotFoundError:
        print(f"Error: The file at {file_path} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
file_path = 'saved_data/cifar_train500/class36.txt'
read_and_print_file(file_path)
